### 将文本分类到标签中

In [1]:
! pip install --upgrade --quiet langchain-core

In [2]:
! pip install -qU "langchain[openai]"

In [19]:
import getpass
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] ="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" #getpass.getpass()

In [22]:
from langchain.chat_models import init_chat_model

llm  = init_chat_model(
    model="Qwen/Qwen2.5-7B-Instruct",
    model_provider="openai",
    api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx", 
    base_url="https://api.siliconflow.cn/v1",
    temperature=0.1,
    max_retries=3,
    
)

In [21]:
llm.invoke("What is the capital of France?")

AIMessage(content='\nThe capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 12, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'THUDM/glm-4-9b-chat', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3ca9dd4-2a1e-440f-ac66-c73058b304dc-0', usage_metadata={'input_tokens': 12, 'output_tokens': 8, 'total_tokens': 20, 'input_token_details': {}, 'output_token_details': {}})

- 让我们在模式中指定一个具有一些属性及其预期类型的​​ Pydantic 模型

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel,Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
从以下段落中提取所需信息。
仅提取“分类”函数中提到的属性。
段落：
{input}
"""
)
class Classification(BaseModel):
    sentiment: str = Field(description="文本的情感倾向。")
    aggressiveness: int = Field(
        description="文本的攻击性在 1 到 10 的范围内是多少？"
    )
    language: str = Field(description="文本所使用的语言")

llm = ChatOpenAI(temperature=0,
        model="Qwen/Qwen2.5-7B-Instruct",
    api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx", 
    base_url="https://api.siliconflow.cn/v1").with_structured_output(Classification)



In [26]:
inp = "我非常高兴认识你！我认为我们会成为很好的朋友！"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

ValidationError: 1 validation error for Classification
  Invalid JSON: EOF while parsing a string at line 1 column 86 [type=json_invalid, input_value='")));提取的属性信...关的任何属性。', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid